<a href="https://colab.research.google.com/github/BrainyHaron/DB-SFD/blob/Brainy's_Projects/Quantile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
# import tensorflow.compat.v1 as tf
import tensorflow as tf
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from tqdm import tqdm

In [ ]:
! pip install tensorflow==2.3.1

ERROR: Could not find a version that satisfies the requirement tensorflow==2.3.1 (from versions: 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0)
ERROR: No matching distribution found for tensorflow==2.3.1


In [ ]:
! python --version

Python 3.10.12


In [ ]:
%matplotlib inline

# Initialize session
# sess = tf.Session()

In [ ]:
from google.colab import drive, files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file «{name}» with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

In [ ]:
from google.colab import drive
drive_path = "/content/drive/MyDrive/Sochi_2023_AI/"

In [ ]:
train_y = np.load(drive_path + "/y_smp_train_scaled.npy")
print(train_y.shape)
train_y = np.reshape(train_y, (train_y.shape[0], train_y.shape[1]*train_y.shape[2]))
print(train_y.shape)
test_y = np.load(drive_path + "/y_smp_test_scaled.npy")
print(test_y.shape)

(1000000, 200, 3)
(1000000, 600)
(100000, 200, 3)


In [ ]:
train_y.shape[1]*train_y.shape[2]

600

In [ ]:
train_pars = np.load(drive_path + "/pars_smp_train.npy")
print(train_pars.shape)
train_pars = np.reshape(train_pars, train_pars.shape[:2])
print(train_pars.shape)

(1000000, 15, 1)
(1000000, 15)


In [ ]:
x = train_y[:250000]

In [ ]:
# # Load data
# mcycle = pd.read_csv('mcycle',delimiter='\t')
# mcycle.times = (mcycle.times - mcycle.times.mean())/mcycle.times.std()
# mcycle.accel = (mcycle.accel - mcycle.accel.mean())/mcycle.accel.std()

# # Reshape to input format for network
# times = np.expand_dims(mcycle.times.values, 1)
# accel = np.expand_dims(mcycle.accel.values, 1)

In [ ]:
# Create network
class q_model:
    def __init__(self,
                 sess,
                 quantiles,
                 in_shape= 600,
                 out_shape=15,
                 batch_size=32):

        self.sess = sess

        self.quantiles = quantiles
        self.num_quantiles = len(quantiles)

        self.in_shape = in_shape
        self.out_shape = out_shape
        self.batch_size = batch_size

        self.outputs = []
        self.losses = []
        self.loss_history = []

        self.build_model()

    def build_model(self, scope='q_model', reuse=tf.AUTO_REUSE):
        with tf.variable_scope(scope, reuse=reuse) as scope:
            tf.disable_eager_execution()


            self.x = tf.placeholder(tf.float32, shape=(None, self.in_shape))
            self.y = tf.placeholder(tf.float32, shape=(None, self.out_shape))

            self.layer0 = tf.layers.dense(self.x,
                                    units = 512,
                                    activation = tf.nn.relu)

            self.layer1 = tf.layers.dense(self.layer0,
                                    units = 256,
                                    activation = tf.nn.relu)

            self.layer2 = tf.layers.dense(self.layer1,
                                    units = 32,
                                    activation = tf.nn.relu)

            # Create outputs and losses for all quantiles
            for i in range(self.num_quantiles):
                q = self.quantiles[i]

                # Get output layers
                output = tf.layers.dense(self.layer2, 15, name="{}_q{}".format(i, int(q*100)))
                self.outputs.append(output)

                # Create losses

                error = tf.subtract(self.y, output)
                loss = tf.reduce_mean(tf.maximum(q * error, (q - 1) * error), axis=-1)

                self.losses.append(loss)

            # Create combined loss
            self.combined_loss = tf.reduce_mean(tf.add_n(self.losses))
            self.train_step = tf.train.AdamOptimizer().minimize(self.combined_loss)

    def fit(self, x, y, epochs=100):
        for epoch in range(epochs):
            epoch_losses = []
            for idx in range(0, x.shape[0], self.batch_size):
                batch_x = x[idx : min(idx + self.batch_size, x.shape[0]),:]
                batch_y = y[idx : min(idx + self.batch_size, y.shape[0]),:]

                feed_dict = {self.x: batch_x,
                             self.y: batch_y}

                _, c_loss = self.sess.run([self.train_step, self.combined_loss], feed_dict)
                epoch_losses.append(c_loss)

            epoch_loss =  np.mean(epoch_losses)
            self.loss_history.append(epoch_loss)
            if epoch % 100 == 0:
                print("Epoch {}: {}".format(epoch, epoch_loss))


    def predict(self, x):
        # Run model to get outputs
        feed_dict = {self.x: x}
        predictions = sess.run(self.outputs, feed_dict)

        return predictions


In [ ]:
# Instantiate model
quantiles = [.1, .25, .5, .75, .9]
model = q_model(sess, quantiles, batch_size=32)

# Initialize all variables
init_op = tf.global_variables_initializer()
sess.run(init_op)

<ipython-input-48-25ca391e85ff>:33: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  self.layer0 = tf.layers.dense(self.x,


ValueError: ignored

In [ ]:
train_y.shape, train_pars.shape

((1000000, 600), (1000000, 15))

In [ ]:
from torch.nn.modules import activation
model_quantile_10 = tf.keras.models.Sequential()
model_quantile_10.add(tf.keras.layers.Dense(512, activation='relu', input_dim = 600))
model_quantile_10.add(tf.keras.layers.Dense(256, activation='relu'))
model_quantile_10.add(tf.keras.layers.Dense(64, activation='relu'))
model_quantile_10.add(tf.keras.layers.Dense(15, activation='linear'))

In [ ]:
def quantile_loss(q, y, y_p):
  e = y - y_p
  return tf.keras.backend.mean(tf.keras.backend.maximum(q*e, (q-1)*e))

In [ ]:
quantile = 0.1
model_quantile_10.compile(optimizer='adam', loss = lambda y, y_p:quantile_loss(quantile, y, y_p))

In [ ]:
model_quantile_10.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               307712    
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 64)                16448     
                                                                 
 dense_3 (Dense)             (None, 15)                975       
                                                                 
Total params: 456463 (1.74 MB)
Trainable params: 456463 (1.74 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model_quantile_10.fit(train_y[:250000], train_pars[:250000], epochs=100, batch_size=32)

Epoch 1/100
7813/7813 [==============================] - 72s 9ms/step - loss: 0.0328
Epoch 2/100
7813/7813 [==============================] - 74s 9ms/step - loss: 0.0322
Epoch 3/100
7813/7813 [==============================] - 74s 9ms/step - loss: 0.0320
Epoch 4/100
7813/7813 [==============================] - 74s 10ms/step - loss: 0.0318
Epoch 5/100
7813/7813 [==============================] - 78s 10ms/step - loss: 0.0317
Epoch 6/100
7813/7813 [==============================] - 76s 10ms/step - loss: 0.0316
Epoch 7/100
7813/7813 [==============================] - 77s 10ms/step - loss: 0.0314
Epoch 8/100
7813/7813 [==============================] - 80s 10ms/step - loss: 0.0314
Epoch 9/100
7813/7813 [==============================] - 74s 10ms/step - loss: 0.0313
Epoch 10/100
4517/7813 [================>.............] - ETA: 30s - loss: 0.0312